In [1]:
!pip3 install --upgrade pip
!pip3 install --upgrade pandas
!pip3 install --upgrade sklearn
!pip3 install --upgrade pymysql
!pip3 install --upgrade sqlalchemy
!pip3 install --upgrade pyodb

     |████████████████████████████████| 2.1 MB 1.6 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 KB 5.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 4.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 KB 7.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 4.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1308 sha256=85142ef717a1d50828de3330eb3b60311d9ebeffd126f97c3f3aac2f60f64b62
  Stored in directory: /root/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
     ━━━━

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 
from keras.models import load_model
import requests
import json
import datetime
import pytz
import pymysql
import warnings
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

In [3]:
# database connection init
conn=pymysql.connect(
    host='sql10.freemysqlhosting.net',
    port=int(3306),
    user='sql10489794',
    passwd='IPELfw5A2X',
    db='sql10489794'
)

In [6]:
# fetch weather data from visualcrossing.com
time_now = datetime.datetime.now(tz=pytz.timezone("Europe/Berlin"))
key = "JPRR7UQ66H89YQ23GM3TYFWBU"
date_start = (time_now - datetime.timedelta(days=2)).strftime("%Y-%m-%d") 
date_stop  = (time_now).strftime("%Y-%m-%d")
result = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/krak%C3%B3w/{date_start}/{date_stop}?unitGroup=metric&include=hours&key={key}&contentType=json")
json_dict = json.loads(str(result.content.decode('utf-8')))
new_dict = {}

# reshape dictionary for further processing
i = 0
for day in json_dict['days'][::-1]:
    for hour in day['hours'][::-1]:
        # leave only relavant data
        if day['datetime'] != time_now.strftime("%Y-%m-%d") or hour['datetime'] < time_now.strftime("%H:%M:00"):
            new_dict[i] = hour.copy()
            # set time in specific format
            new_dict[i]['datetime'] = day['datetime'] + 'T' + hour['datetime']
            # fetch weather measurements from arduino database
            query = f"SELECT * FROM esp WHERE weather_time = '{day['datetime']}' AND hour = '{hour['datetime'].split(':')[0]}'"
            esp_measurement=pd.read_sql_query(query, conn)
            if not esp_measurement.empty:
                print(esp_measurement)
                new_dict[i]['temp'] = esp_measurement['temperature'][0]
                new_dict[i]['humidity'] = esp_measurement['humidity'][0]
            i=i+1

# change shape            
measured_data_wlabels = pd.DataFrame.from_dict(new_dict).transpose()[:48][::-1]

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0  22           21        44    16   2022-05-05


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0  21           21        41    15   2022-05-05


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


...

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


   id  temperature  humidity  hour weather_time
0   3           16        39    21   2022-05-04


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 conne

   id  temperature  humidity  hour weather_time
0   1           21        41    22   2022-05-03


/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 conne

In [7]:
measured_data_wlabels

,datetime,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,source
47,2022-05-03T18:00:00,1651593600,18.0,18.0,42.25,4.9,0.0,0.0,0.0,0.0,...,10.0,25.0,467.0,1.7,5.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs
46,2022-05-03T19:00:00,1651597200,17.0,17.0,48.19,5.9,0.0,0.0,0.0,0.0,...,10.0,25.0,89.0,0.3,1.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs
45,2022-05-03T20:00:00,1651600800,16.0,16.0,47.91,4.9,0.0,0.0,0.0,0.0,...,10.0,78.0,2.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-day,"[EPKT, EPKK, E6527]",obs
44,2022-05-03T21:00:00,1651604400,15.9,15.9,48.16,4.9,0.0,0.0,0.0,0.0,...,10.0,50.0,2.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
43,2022-05-03T22:00:00,1651608000,21,13.9,41,5.9,0.0,0.0,0.0,0.0,...,10.0,88.0,2.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
42,2022-05-03T23:00:00,1651611600,13.9,13.9,58.77,5.9,0.0,0.0,0.0,0.0,...,10.0,88.0,2.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
41,2022-05-04T00:00:00,1651615200,12.9,12.9,67.05,6.9,0.0,0.0,0.0,0.0,...,10.0,25.0,3.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
40,2022-05-04T01:00:00,1651618800,10.9,10.9,76.42,6.9,0.0,0.0,0.0,0.0,...,10.0,50.0,3.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
39,2022-05-04T02:00:00,1651622400,10.9,10.9,71.34,5.9,0.0,0.0,0.0,0.0,...,10.0,50.0,3.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs
38,2022-05-04T03:00:00,1651626000,9.9,9.6,76.18,5.9,0.0,0.0,0.0,0.0,...,10.0,78.4,3.0,0.0,0.0,10.0,Partially cloudy,partly-cloudy-night,"[EPKT, EPKK, E6527]",obs


In [8]:
columns_names = [col for col in measured_data_wlabels]

pd.DataFrame(columns_names, columns=['column_name'])

,column_name
0,datetime
1,datetimeEpoch
2,temp
3,feelslike
4,humidity
5,dew
6,precip
7,precipprob
8,snow
9,snowdepth


In [10]:
import joblib
sc = joblib.load("scaler.save")

In [11]:
selected_columns = [2, 4, 21]
selected_columns_names = [columns_names[col] for col in selected_columns]
print(selected_columns_names)

measured_data_wlabels = measured_data_wlabels.iloc[:,selected_columns]

# perform one-hot encoding of weather conditions
cond = ['Snow', 'Rain, Partially cloudy', 'Snow, Partially cloudy', 'Rain', 'Partially cloudy', 'Rain, Overcast', 'Snow, Rain, Overcast', 'Overcast', 'Snow, Overcast', 'Clear', 'Snow, Rain, Partially cloudy']
for label in cond:
    measured_data_wlabels[label] = 0
    measured_data_wlabels.loc[measured_data_wlabels['conditions']==label, label] = 1
measured_data_wlabels = measured_data_wlabels.drop(columns='conditions')

measured_data = measured_data_wlabels.values

measured_data = sc.transform(measured_data)
measured_data = np.array(measured_data)
measured_data = np.reshape(measured_data,(1,measured_data.shape[1],measured_data.shape[0]))

['temp', 'humidity', 'conditions']


In [12]:
!tar xzf weather_forecast_model.tgz

In [13]:
model = load_model('weather_forecast_model')

In [14]:
# predict weather with precomputed model
predicted_weather = model.predict(measured_data)
predicted_weather = np.reshape(predicted_weather, (predicted_weather.shape[2],predicted_weather.shape[1]))
predicted_weather = sc.inverse_transform(predicted_weather)

In [15]:
# restore conditions from one-hot encoding
predicted_weather_wlabels = pd.DataFrame(predicted_weather, columns=[col for col in selected_columns_names if col != 'conditions']+cond)
max_cond = np.argmax(predicted_weather[:, 2:], axis=1)
for i, mcond in enumerate(max_cond):
    predicted_weather_wlabels.loc[i, 'conditions'] = cond[mcond]
predicted_weather_wlabels = predicted_weather_wlabels.drop(columns=cond)


In [16]:
# add datetime for storing purposes
predicted_weather_wlabels['datetime'] = [(time_now + datetime.timedelta(hours=i+1)).strftime("%Y-%m-%dT%H:00:00") for i in range(12)]
column_to_move = predicted_weather_wlabels.pop("datetime")
predicted_weather_wlabels.insert(0, "datetime", column_to_move)
predicted_weather_wlabels

,datetime,temp,humidity,conditions
0,2022-05-05T18:00:00,15.654381,61.519917,Partially cloudy
1,2022-05-05T19:00:00,13.826118,61.527149,Partially cloudy
2,2022-05-05T20:00:00,14.680547,60.780479,Partially cloudy
3,2022-05-05T21:00:00,13.368874,60.531212,Partially cloudy
4,2022-05-05T22:00:00,11.636752,62.609039,Partially cloudy
5,2022-05-05T23:00:00,10.239193,66.587532,Partially cloudy
6,2022-05-06T00:00:00,11.413772,66.489365,Partially cloudy
7,2022-05-06T01:00:00,10.886472,58.532631,Partially cloudy
8,2022-05-06T02:00:00,10.744816,61.505550,Partially cloudy
9,2022-05-06T03:00:00,10.034091,69.505104,Clear


In [17]:
# push forecast to remote database
cursor = conn.cursor()
for row in predicted_weather_wlabels[::-1].iterrows():
    date, hour = row[1]['datetime'].split("T")[0], row[1]['datetime'].split("T")[1]
    temp = row[1]['temp']
    humidity = row[1]['humidity'] 
    conditions = row[1]['conditions'] 
    query = f"INSERT INTO forecast (date, hour, temp, humidity, conditions) VALUES ('{date}', '{hour}', {int(temp)}, {int(humidity)}, '{conditions}')"
    cursor.execute(query)
    conn.commit()